In [2]:
import itertools

import pandas as pd
import numpy as np
import pynetlogo
from tqdm import tqdm

### Connect to NetLogo

In [9]:
import os, sys
if sys.platform == 'win32':
    os.environ["JAVA_HOME"] = 'C:/Program Files/NetLogo 6.3.0/runtime/bin/server/'


netlogo = pynetlogo.NetLogoLink(gui=False)
netlogo.load_model('Altruism_RMAI.nlogo')

### Define Experiment Parameters

In [12]:
# # length of each experiment
# TICKS = 10

# # how many to run each experiment
# N_RUNS = 1

# experiment_parameters = {
#     'occupation-prob': [0.1, 0.5, 0.9],
#     'occupation-radius': [10, 20, 40],
#     'occupation-diffusion': [0.2, 0.6, 1],
#     'prob-gain-resource': [0.1, 0.5, 0.9],
#     'agent-move-cost': [1, 40, 80],
#     'stride-length': [0.1, 0.7, 0.8],
#     'amt-resource-consumption': [20, 60, 99],
#     'altruism-resource-threshold': [1, 80, 99],
#     'reproduction-threshold': [1, 80, 99],
#     'reproduction-cost': [1, 60, 99]
# }

To reduce the parameter space, we adjust parameters in groups of related simulation variables. 

- **occupation**: variable with range $[0, 1]$, where we assume that the probability, radius, and smoothness (diffusion) of the occupied region is linear. This maps to: 
  - *occupation-prob* with range $[0, 1]$
  - *occupation-radius* with range $[0, 40]$ 
  - *occupation-diffusion* with range $[0, 1]$
- **initial-harshness**: variable with range $[0, 50]$ that corresponds to the initial harshness of the patches, and is related to the amount of occupation.
- **viscosity**: variable with range $[0, 1]$, where we assume that the cost of dispersal is higher when the movement distance is longer. This is inversely proportional to the following ranges 
  - *agent-move-cost* with range $[0, 20]$
  - *stride-length* with range $[0, 1]$
- **consumption**: variable with range $[0, 1]$, where we assume that there is a inverse relation between the probability of a patch gaining new resources and the amount of energy that an agent gains after consuming resources. This maps to
  - *prob-gain-resource* with range $[0, 1]$
  - *energy-gain* with range $[0, 100]$
- **reproduction**: variable with range $[0, 1]$, where we assume that the energy threshold to reproduce and the cost to reproduce are correlated. This cost should have a tighter range, so that agents do not immediately die after reproducing. This maps to
  - *reproduction-threshold* with range $[50, 200]$
  - *reproduction-cost* with range $[50, 100]$
- **altruism-factor**: variable with range $[0, 100]$ that corresponds to the amount of altruism that altruistic agents confer by being conservative about resource consumption, in effect leaving more resources to other agents.

In [10]:
# length of each experiment
TICKS = 100

# amount of values to consider per param
LEVELS = 5

# how many to run each experiment
N_RUNS = 10

experiment_parameters = {
    'occupation': np.linspace(0, 1, LEVELS),
    'initial-harshness': np.linspace(0, 1, LEVELS),
    'viscosity': np.linspace(0, 1, LEVELS),
    'consumption': np.linspace(0, 1, LEVELS),
    'reproduction': np.linspace(0, 1, LEVELS),
    'altruism-factor': np.linspace(0, 1, LEVELS),
}

param_dict = {
    'occupation': {
        'occupation-prob': (0, 1),
        'occupation-radius': (0, 40),
        'occupation-diffusion': (0, 1),
    },
    'initial-harshness': {'initial-patch-harshness': (0, 50)},
    'viscosity': {
        'agent-move-cost': (20, 0),
        'stride-length': (1, 0)
    },
    'consumption': {
        'prob-gain-resource': (0, 1),
        'energy-gain': (100, 0)
    },
    'reproduction': {
        'reproduction-threshold': (50, 200),
        'reproduction-cost': (50, 100)
    },
    'altruism-factor': {'altruism-resource-threshold': (0, 100)},
}

def mapsto(x, params: dict[str, tuple[int]]):
    return {key: x*(value[1]-value[0])+value[0] for key, value in params.items()}

mapsto(0.8, param_dict['consumption'])

{'prob-gain-resource': 0.8, 'energy-gain': 20.0}

### Make list of all parameter combinations

In [11]:
keys, values = zip(*experiment_parameters.items())
parameter_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
# parameter_combinations = [param for param in parameter_combinations for _ in range(0, N_RUNS)]
len(parameter_combinations)

15625

### Run experiments

In [31]:
results = []
TICKS = 20

for params in tqdm(parameter_combinations):
    netlogo.command('setup')

    for param, value in params.items():
        mapped_params = mapsto(value, param_dict[param])
        for mapped_param, mapped_value in mapped_params.items():
            netlogo.command(f'set {mapped_param} {mapped_value}')
    
    # counts = netlogo.repeat_report(['count altruism-agents', 'count greedy-agents'], TICKS)
    # results.append({
    #     **params, 
    #     'Altruism Agents': np.mean(counts['count altruism-agents']), 
    #     'Greedy Agents': np.mean(counts['count greedy-agents']), 
    #     'Number of ticks': counts['ticks'][-1]
    # })

    # print(results)
    # print(counts)
    
    # break

    netlogo.command(f'repeat {TICKS} [go if count turtles = 0 [stop]]')
    # Collect results
    count_altruism_agents = netlogo.report('count altruism-agents')
    count_greedy_agents = netlogo.report('count greedy-agents')
    n_ticks = netlogo.report('ticks')

    results.append({**params, 'Altruism Agents': count_altruism_agents, 'Greedy Agents': count_greedy_agents, 'Number of ticks': n_ticks})
    

  0%|          | 0/15625 [00:00<?, ?it/s]

43004.0


In [32]:
n_ticks

10.0

In [7]:
results_df = pd.DataFrame(results)

In [8]:
results_df

,occupation,initial-harshness,viscosity,consumption,reproduction,altruism-factor,Altruism Agents,Greedy Agents,Number of ticks
0,0.0,0.0,0.0,0.0,0.00,0.00,89.554455,109.455446,100.0
1,0.0,0.0,0.0,0.0,0.00,0.25,91.544554,107.465347,100.0
2,0.0,0.0,0.0,0.0,0.00,0.50,93.534653,105.475248,100.0
3,0.0,0.0,0.0,0.0,0.00,0.75,69.653465,129.356436,100.0
4,0.0,0.0,0.0,0.0,0.00,1.00,89.554455,109.455446,100.0
5,0.0,0.0,0.0,0.0,0.25,0.00,103.485149,95.524752,100.0
6,0.0,0.0,0.0,0.0,0.25,0.25,101.495050,97.514851,100.0
7,0.0,0.0,0.0,0.0,0.25,0.50,107.465347,91.544554,100.0
8,0.0,0.0,0.0,0.0,0.25,0.75,93.534653,105.475248,100.0
9,0.0,0.0,0.0,0.0,0.25,1.00,97.514851,101.495050,100.0


### Concurrent implementation